In [2]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# session
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_ = tf.placeholder(tf.float32,shape=[None,10])

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
sess.run(tf.global_variables_initializer())

In [15]:
# Build a multilayer convnet

# Weight Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

# The first two dimensions are the patch size, 
# the next is the number of input channels, and 
# the last is the number of output channels.

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [16]:
# Convolution and Pooling
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2X2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [19]:
# First Conv layer
W_conv1 = weight_variable([5,5,1,32])

# To apply the layer, we first reshape x to a 4d tensor, 
# with the second and third dimensions corresponding to image 
# width and height, and the final dimension corresponding to the 
# number of color channels

b_conv1 = bias_variable([32])

In [21]:
x_image = tf.reshape(x,[-1,28,28,1])

In [22]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2X2(h_conv1)

In [24]:
# Second Conv layer
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2X2(h_conv2)

In [25]:
# Densely Connected layer
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

In [26]:
# Dropout 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [27]:
# Readout layer
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

In [28]:
# train
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                labels=y_,logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [34]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(0,20000):
             batch = mnist_data.train.next_batch(50)
             if(i%100 == 0):
                    train_accuracy = accuracy.eval(feed_dict={
                         x:batch[0],y_:batch[1],keep_prob:1.0
                     })
                    print('Step %d, training accuracy %g' % (i,train_accuracy))
                    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
    
    print('Test accuracy %g' % (accuracy.eval(feed_dict={
        x:mnist_data.test.images,y_:mnist_data.test.labels,keep_prob:1.0
    })))
             
    
      

Step 0, training accuracy 0.16
Step 100, training accuracy 0.2
Step 200, training accuracy 0.12
Step 300, training accuracy 0.2
Step 400, training accuracy 0.2
Step 500, training accuracy 0.26
Step 600, training accuracy 0.18
Step 700, training accuracy 0.28
Step 800, training accuracy 0.32
Step 900, training accuracy 0.3
Step 1000, training accuracy 0.24
Step 1100, training accuracy 0.2
Step 1200, training accuracy 0.16
Step 1300, training accuracy 0.32
Step 1400, training accuracy 0.42
Step 1500, training accuracy 0.42
Step 1600, training accuracy 0.44
Step 1700, training accuracy 0.32
Step 1800, training accuracy 0.48
Step 1900, training accuracy 0.38
Step 2000, training accuracy 0.42
Step 2100, training accuracy 0.52
Step 2200, training accuracy 0.54
Step 2300, training accuracy 0.56
Step 2400, training accuracy 0.54
Step 2500, training accuracy 0.6
Step 2600, training accuracy 0.48
Step 2700, training accuracy 0.68
Step 2800, training accuracy 0.54
Step 2900, training accuracy 0.5